## CS431/631 Big Data Infrastructure
### Winter 2018 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Yangjie Zhou_
* **ID:** _20744733_

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://lintool.github.io/bigdata-2018w/assignment1-431.html).

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located.   You will be using the Spark installation in the CS451 course account.   The code in the following cell tells Python how to find this Spark installation.   Before going on, execute that code (by selecting the cell and hitting 'return' while holding down the shift key).   It will take a few seconds to run, and will produce no output.

In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster (since we do not have a Spark cluster in the CS student computing environment), and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Run the code in the cell below to create a Spark context.   Creating the `SparkContext` causes your Python program (running in this notebook) to prepare to run Spark jobs, and will take a few seconds to complete.  Be sure that you run this code only one time, because a single Python program may only have one active SparkContext.

In [2]:
sc = SparkContext(appName="YourTest", master="local[2]")

Next, let's test that your `SparkContext` has been set up properly by running some simple test code (adapted from the [Spark examples page](https://spark.apache.org/examples.html)).   This code uses a single Spark job to estimate the value of $\pi$.  `parallelize()` and `filter()` are Spark *transformations*, and `count()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take a few seconds, since a Spark job is being created and executed, and should print an estimate of $\pi$ when it finishes.   

In [3]:
import random

num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi_estimate = 4 * count / num_samples
print(pi_estimate)

3.14173232


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the $\pi$-estimation example works.   What is the Spark job doing, and how is it used to estimate the value of $\pi$?

#### Your answer to Question 1:

*The transformation **parallelize()** claim the interger list of length 100000000 to be the RDD, of which each interger can be regarded as the index of each sample.* 

*The following transformation **filter()** will just keep the point as the function **inside** return TRUE, Defining the dependency between the son RDD with the father RDD.*

*The final action **count()** will compute how many points are left after the **filter()**, activating the program to process.*

___

* *This estimation uses the Marto Carlo method for the integral operation of the quadrntnt by the $count/num\_sample$. Finally, according to formula for the aera of circle $\pi r^2$ where r is equal to 1, the $\pi$ is equal to the 4 times the estimated quadrntnt area.*

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs.   Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`), and it should print the number of distinct tokens.

In [4]:
# your solution to Question 2 here
from simple_tokenize import simple_tokenize
from operator import add

text=sc.textFile("Shakespeare.txt")
counts=text.flatMap(simple_tokenize) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(add).count()
print(counts)

25975


---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1.   Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`), and it should print the number of distinct token pairs.

In [5]:
# your solution to Question 3 here
from simple_tokenize import simple_tokenize
from itertools import permutations
from operator import add

def simple_pairize(line):
    t = simple_tokenize(line)
    pairs=[]
    for pair in permutations(set(t), 2):
        pairs.append(pair)
    return pairs
    
text=sc.textFile("Shakespeare.txt")
counts=text.flatMap(simple_pairize) \
             .map(lambda pair: (pair, 1)) \
             .reduceByKey(add).count()
print(counts)

1969760


---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate $n(x)$ and $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your code should report (print) the 50 highest-probability tokens, and their probabilities.

Make sure that your solution calculates $n(x)$ and $p(x)$ and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their probabilities) should be returned by Spark to your driver code.

Write your solution for in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`), and it should print the 50 highest-probability tokens, along with their counts ($n(x)$) and probabilities ($p(x)$).

In [6]:
# your solution to Question 4 here
from simple_tokenize import simple_tokenize
from operator import add

text=sc.textFile("Shakespeare.txt")

num=text.flatMap(simple_tokenize) \
             .count()
tokens=text.flatMap(simple_tokenize) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(add) \
             .mapValues(lambda x: (x,x/num)) \
             .sortBy(lambda x:x[1][1],False).take(50)
for i in tokens:
    print(i)


('the', (27378, 0.030855264937383355))
('and', (26082, 0.029394660679992426))
('i', (20717, 0.023348254938555444))
('to', (19661, 0.022158132951051724))
('of', (17473, 0.01969223625724667))
('a', (14723, 0.0165929602481224))
('you', (13630, 0.015361138910677738))
('my', (12490, 0.014076348128713495))
('in', (10996, 0.01239259599866562))
('that', (10915, 0.012301308232578688))
('is', (9137, 0.010297485416497614))
('not', (8512, 0.009593104505333008))
('with', (7778, 0.008765879563261294))
('me', (7777, 0.00876475255380343))
('it', (7692, 0.008668956749885045))
('for', (7578, 0.00854047767168862))
('be', (6867, 0.007739173947147764))
('his', (6859, 0.007730157871484858))
('your', (6657, 0.007502501960996457))
('this', (6606, 0.007445024478645425))
('but', (6277, 0.007074238367008376))
('he', (6260, 0.0070550792062246985))
('have', (5885, 0.0066324506595259345))
('as', (5744, 0.0064735423259672))
('thou', (5491, 0.006188408933127767))
('him', (5205, 0.005866084228178843))
('so', (5056, 0.

---
#### Question 5  (6/30 marks):

Next, write Spark code that will prompt the user to input a positive integer threshold $T$, and then print all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the program should also report $n(x,y)$ and PMI$(x,y)$.    You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

As always, calculations should be done in data-parallel fashion by using Spark.

In [7]:
# your solution to Question 5 here
from simple_tokenize import simple_tokenize
from itertools import permutations
from operator import add
from math import log

def simple_pairize(line):
    t = simple_tokenize(line)
    pairs=[]
    for pair in permutations(set(t), 2):
        pairs.append(pair)
    return pairs

def PMI(x):
    p=(x[1]/num_pairs)/(tokens[x[0][0]]*tokens[x[0][1]])
    return (x[0],(x[1],log(p,10)))

while True:
    try:
        threshold = input("Input a positive integer frequency threshold(press return to exit): ")
        if len(threshold) == 0:
            break
        if int(threshold) < 0:
            continue
        else:
            threshold=int(threshold)
    except ValueError:
        print("Threshold must be a positive integer!")
        continue
    
    # Spark park    
    text=sc.textFile("Shakespeare.txt")
    
    num_tokens=text.flatMap(simple_tokenize).count()
    num_pairs=text.flatMap(simple_pairize).count()
    
    tokens=text.flatMap(simple_tokenize) \
                 .map(lambda word: (word, 1)) \
                 .reduceByKey(add).mapValues(lambda x: x/num_tokens).collectAsMap()
    
    pairs=text.flatMap(simple_pairize) \
         .map(lambda pair: (pair, 1)) \
         .reduceByKey(add)\
        .filter(lambda x:x[1]>=threshold)\
        .map(PMI)\
        .sortByKey()\
        .collect()
    for pair in pairs:
        print(pair)

Input a positive integer frequency threshold(press return to exit): 2500
(('a', 'and'), (2672, -0.06692767769672042))
(('and', 'a'), (2672, -0.06692767769672042))
(('and', 'i'), (3338, -0.1186083531155394))
(('and', 'of'), (3565, -0.016075762136901896))
(('and', 'the'), (5427, -0.028609716990176635))
(('and', 'to'), (3815, -0.037878880315267205))
(('i', 'and'), (3338, -0.1186083531155394))
(('i', 'my'), (2586, 0.09031228728595314))
(('i', 'the'), (3058, -0.1777180329642898))
(('i', 'to'), (3095, -0.028698744720967992))
(('i', 'you'), (2854, 0.09520431805966799))
(('in', 'the'), (2863, 0.06875796885499567))
(('my', 'i'), (2586, 0.09031228728595314))
(('of', 'and'), (3565, -0.016075762136901896))
(('of', 'the'), (7266, 0.27209926862008577))
(('the', 'and'), (5427, -0.028609716990176635))
(('the', 'i'), (3058, -0.1777180329642898))
(('the', 'in'), (2863, 0.06875796885499567))
(('the', 'of'), (7266, 0.27209926862008577))
(('the', 'to'), (4072, -0.03062648185928795))
(('to', 'and'), (3815, 

## ---
#### Question 6  (6/30 marks):

Finally, write Spark code that will prompt the user for two inputs: a positive integer threshold $T$, and a sample size $N$.   For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair.

For each $x$, the output of your program should be similar to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$ - except that here you report all co-occuring tokens above the threshold, rather than just five.   Rather than producing output for all possible tokens $x$, your program should produce output only for $N$ different $x$'s, chosen *at random* from among all distinct tokens in the input file.


In [ ]:
# your solution to Question 6 here
from simple_tokenize import simple_tokenize
from itertools import permutations
from operator import add
from math import log

def simple_pairize(line):
    t = simple_tokenize(line)
    pairs=[]
    for pair in permutations(set(t), 2):
        pairs.append(pair)
    return pairs

def PMI(x):
    p=(x[1]/num_pairs)/(tokens[x[0][0]]*tokens[x[0][1]])
    return (x[0][0],(x[0][1],x[1],log(p,10)))

def f(x):
    y=[]
    for item in x:
        if item[1]>=threshold:
            y.append(item)
    return y

while True:
    try:
        threshold = input("Input a positive integer frequency threshold(press return to exit): ")
        if len(threshold) == 0:
            break
        N = input("Input a positive integer sample size N: ")
        if int(threshold) < 0 or int(N)<0:
            continue
        else:
            threshold=int(threshold)
            N=int(N)
    except ValueError:
        print("Threshold and N must be a positive integer!")
        continue
    
    # Spark park    
    text=sc.textFile("Shakespeare.txt")
    
    num_tokens=text.flatMap(simple_tokenize).count()
    num_pairs=text.flatMap(simple_pairize).count()
    
    tokens=text.flatMap(simple_tokenize) \
                 .map(lambda word: (word, 1)) \
                 .reduceByKey(add).mapValues(lambda x: x/num_tokens).collectAsMap()
    
    pairs=text.flatMap(simple_pairize) \
        .map(lambda pair: (pair, 1)) \
        .reduceByKey(add)\
        .map(PMI)\
        .groupByKey()\
        .mapValues(f)\
        .takeSample(False,N,seed=123)
        
    result=sorted([(x, sorted(y)) for (x, y) in pairs])
    for item in result:
        print(item)
        if len(item[1])>0:
            for i in item[1]:
                print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(\
                    item[0], i[0], i[1],i[2]),"\n")
        else:
            print("    {0} don't have tokens co-occured with at least {1} times".format(\
                    item[0], threshold),"\n")

Input a positive integer frequency threshold(press return to exit): 20
Input a positive integer sample size N: 10
('beg', [('and', 26, 0.11148192326862193), ('i', 34, 0.32800151686912077), ('to', 31, 0.3106055565137391)])
    n(beg,and) = 26,  PMI(beg,and) = 0.11148192326862193 

    n(beg,i) = 34,  PMI(beg,i) = 0.32800151686912077 

    n(beg,to) = 31,  PMI(beg,to) = 0.3106055565137391 

('coarse', [])
    coarse don't have tokens co-occured with at least 20 times 

('cushes', [])
    cushes don't have tokens co-occured with at least 20 times 

('faction', [])
    faction don't have tokens co-occured with at least 20 times 

('forest', [('of', 24, 0.37715848724222506), ('the', 54, 0.534306763108721)])
    n(forest,of) = 24,  PMI(forest,of) = 0.37715848724222506 

    n(forest,the) = 54,  PMI(forest,the) = 0.534306763108721 

('hound', [])
    hound don't have tokens co-occured with at least 20 times 

('philippan', [])
    philippan don't have tokens co-occured with at least 20 times 

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.